In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pymysql
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score
from xgboost import XGBClassifier
import seaborn as sns
import plotly.graph_objects as go


In [2]:
df=pd.read_excel('female_players.xlsx')

In [3]:
def high_correlated_cols(dataframe, plot=False, corr_th=0.80):
    # Select only the numeric columns from the DataFrame
    numeric_dataframe = dataframe.select_dtypes(include=['number'])
    
    corr = numeric_dataframe.corr()
    cor_matrix = corr.abs()
    upper_triangle_matrix = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(bool))
    drop_list = [col for col in upper_triangle_matrix.columns if any(upper_triangle_matrix[col] > corr_th)]
    
    if plot:
        fig = go.Figure(data=go.Heatmap(
                       z=corr.values,
                       x=corr.columns,
                       y=corr.index,
                       colorscale='RdBu',
                       colorbar=dict(title='Correlation'),
                       zmin=-1, zmax=1))
        
        fig.update_layout(title='Correlation Matrix',
                          xaxis=dict(title='Columns'),
                          yaxis=dict(title='Columns'))
        
        # Explicitly show the figure
        fig.show()
    
    return drop_list

In [4]:
high_correlated_cols(df)

['Overall',
 'Sprint',
 'Positioning',
 'Finishing',
 'Shot',
 'Long',
 'Volleys',
 'Penalties',
 'Vision',
 'Crossing',
 'Free',
 'Curve',
 'Reactions',
 'Ball',
 'Composure',
 'Interceptions',
 'Def',
 'Standing',
 'Sliding',
 'Stamina',
 'Strength',
 'Skill moves',
 'GK']

In [5]:

class DatabaseCreator:
    def create_mysql_connection(self, hostname, username, password, database):
        # Connect to MySQL database
        # conn = pymysql.connect(
        #     host=hostname,
        #     user=username,
        #     password=password,
        #     database=database
        # )
        
        # return conn
        engine = create_engine(f'mysql+pymysql://{username}:{password}@{hostname}/{database}')
        return engine
    def add_excel_to_sql(self,conn,file):
        x1=pd.ExcelFile(file)
        for sheet in x1.sheet_names:
            df=pd.read_excel(x1,sheet)
            df.to_sql(sheet,conn,if_exists='replace',index=False)
    def add_dataframe_to_sql(self, conn, df, table_name):
        try:
            if df is None:
                raise ValueError("DataFrame is None")
            
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"DataFrame successfully added to table '{table_name}' in the database.")
        except Exception as e:
            print(f"Error adding DataFrame to SQL: {e}")
        
    def show_query_results(self, conn, queries):
        """
        Executes SQL queries and displays the results.

        Args:
        - conn: Database connection object
        - queries: Dictionary containing sheet names as keys and SQL queries as values
        """
        for sheet_name, query in queries.items():
            print(f"Results for query in sheet '{sheet_name}':")
            df = pd.read_sql_query(query, conn)
            print(df)
            print()

    def fetch_data_to_excel(self, conn, queries, output_file):
        with pd.ExcelWriter(output_file, mode='a') as writer:
            for sheet_name, query in queries.items():
                # Read SQL query results into DataFrame
                df = pd.read_sql_query(query, conn)

                # Write DataFrame to Excel sheet
                df.to_excel(writer, sheet_name=sheet_name, index=False)



In [6]:

print("add your hostname:")
val1=int(input())
print(val1)
hostname=val1
print("add your username:")
val2=int(input())
print(val2)
username=val2
print("add your password:")
val3=int(input())
print(val3)
password=val3
print("add your database:")
val4=int(input())
print(val4)
database=val4
# Connect to MySQL database
conn = creator.create_mysql_connection(hostname, 
                                       username, 
                                       password, 
                                       database)

In [7]:
creator.add_dataframe_to_sql(conn,df,'female_players')

DataFrame successfully added to table 'female_players' in the database.


In [12]:


class FootballStylePredictor:
    print("Type Object.start() to start the class")
    
    def __init__(self):
        self.final_male_input = None
        self.final_male_output = None
        self.gk_male_input = None
        self.gk_male_output = None
        self.models_final_male = []
        self.models_gk_male = []
        self.predictions = {}  # Store predictions as a dictionary

    def intake_data(self, file1, file2):
        # Read data from CSV files
        final_male = pd.read_csv(file1)
        gk_male = pd.read_csv(file2)

        # Drop unnecessary columns
        final_male.drop(columns=['ranking', 'Name', 'Nation', 'Club', 'Club_Country'], inplace=True)
        gk_male.drop(columns=['ranking', 'Name', 'Nation', 'Club', 'Club_Country'], inplace=True)

        # Mapping dictionaries for categorical values
        positions = {'ST': 9.00, 'CM': 5.00, 'CF': 11.00, 'RW': 10.00, 'CB': 3.00, 'LW': 7.00, 'CDM': 5.00,
                     'CAM': 12.00, 'RM': 6.00, 'LB': 2.00, 'RB': 4.00, 'LM': 8.00, 'LWB': 13.00, 'RWB': 14.00}

        # Apply transformations and mappings
        final_male['Position'] = final_male['Position'].map(positions)
        
        # Separate input and output data
        self.final_male_output = final_male[['Position']]
        self.gk_male_output = gk_male[['impact']]
        self.final_male_input = final_male.drop(columns=['Position'])
        self.gk_male_input = gk_male.drop(columns=['impact'])

    def prediction(self):
        print('Enter the number: 1=Not a Goalkeeper, 2=Goalkeeper')
        number = int(input())
        models = []
        if number == 1:
            # Train/test split for non-goalkeepers
            X_train, X_test, y_train, y_test = train_test_split(self.final_male_input, self.final_male_output,
                                                                test_size=0.3, random_state=42)
            targets = ['Position']
            best_model = None
            best_recall = 0.0
            y_train_dict = {target: y_train[target] for target in targets}
            
            # Iterate over targets
            for target in targets:
                # Define the parameter grid for grid search
                param_grid = {
                    'penalty': ['l1', 'l2'],
                    'C': [0.001, 0.01, 0.1, 1, 10, 100],
                    'multi_class': ['auto', 'ovr', 'multinomial']
                }

                # Initialize the Logistic Regression model
                log_reg = LogisticRegression(max_iter=1000)
                
                # Perform grid search with cross-validation
                grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, scoring='recall_macro')
                grid_search.fit(X_train, y_train_dict[target])

                # Get the best model from grid search
                best_model = grid_search.best_estimator_

                # Predict on the test set
                y_pred = best_model.predict(X_test)

                # Evaluate the model
                recall = recall_score(y_test[target], y_pred, average='macro')

                # Store the best model and its recall score
                if recall > best_recall:
                    best_recall = recall
                    best_model = best_model

            if best_model:
                models.append(best_model)

            self.models_final_male = models
        elif number == 2:
            # Train/test split for goalkeepers
            X_train, X_test, y_train, y_test = train_test_split(self.gk_male_input, self.gk_male_output,
                                                                test_size=0.3, random_state=42)
            targets = ['impact']
            
            y_train_dict = {target: y_train[target] for target in targets}
            for target in targets:
                model = LinearRegression()
                model.fit(X_train, y_train_dict[target])
                models.append(model)
            self.models_gk_male = models
        else:
            print("Invalid number. Please enter either 1 or 2.")
            return None

        # Store predictions in a dictionary with keys 'prediction_1' and 'prediction_2'
        self.predictions[f'prediction_{number}'] = {target: model.predict(X_test) for target, model in zip(targets, models)}
        return self.predictions

    def add_your_numbers(self):
        print('Enter the number: 1=Not a Goalkeeper, 2=Goalkeeper')
        number = int(input())
        if number == 1:
            self.final_male_input = pd.DataFrame(self.final_male_input)
            max_values_str = "Max value of the given data: "
            max_values = self.final_male_input.max(axis=0)
            print("Max value of the given data, please type your value below the respective attribute:")
            for col, max_val in max_values.items():
                max_values_str += f"{col}: {max_val}, "
            print(max_values_str[:-2])

            inputs = []
            for col in self.final_male_input.columns:
                val = float(input(f"Enter value for {col}: "))
                inputs.append(val)

            predictions = self.predict_with_given_inputs(inputs, 'final_male')
            self.predictions[f'prediction_{number}'] = predictions
            return predictions
        
        elif number == 2:
            self.gk_male_input = pd.DataFrame(self.gk_male_input)
            max_values_str = "Max value of the given data: "
            max_values = self.gk_male_input.max(axis=0)
            print("Max value of the given data, please type your value below the respective attribute:")
            for col, max_val in max_values.items():
                max_values_str += f"{col}: {max_val}, "
            print(max_values_str[:-2])

            inputs = []
            for col in self.gk_male_input.columns:
                val = float(input(f"Enter value for {col}: "))
                inputs.append(val)

            predictions = self.predict_with_given_inputs(inputs, 'gk_male')
            self.predictions[f'prediction_{number}'] = predictions
            return predictions
        
        else:
            print("Invalid number. Please enter either 1 or 2.")

    def predict_with_given_inputs(self, inputs, player_type):
        if player_type == 'final_male':
            models = self.models_final_male
            targets = ['Position']
            inputs_array = np.array(inputs)
            inputs_reshaped = inputs_array.reshape(1, -1)
            predictions = {}
            for target, model in zip(targets, models):
                prediction = model.predict(inputs_reshaped)
                predictions[target] = prediction
            print("Please see the ideal position from this list: ")
            print("'ST': Striker, 'CM': Central Midfielder, 'CF': Center Forward, 'RW': Right Winger, 'CB': Center Back, 'LW': Left Winger, 'CDM': Central Defensive Midfielder, 'CAM': Central Attacking Midfielder, 'RM': Right Midfielder, 'LB': Left Back, 'RB': Right Back, 'LM': Left Midfielder, 'LWB': Left Wing Back, 'RWB': Right Wing Back")
        
        elif player_type == 'gk_male':
            models = self.models_gk_male
            targets = ['impact']
            inputs_array = np.array(inputs)
            inputs_reshaped = inputs_array.reshape(1, -1)
            predictions = {}
            for target, model in zip(targets, models):
                prediction = model.predict(inputs_reshaped)
                predictions[target] = prediction
            print("If Impact less than or equal to 11: Counter Attacking Style Football\nIf Impact more than 11: Attacking Style Football")
        else:
            print("Invalid player type.")
            return None

        return predictions

    def start(self, file1, file2):
        self.intake_data(file1, file2)
        self.prediction()
        print("Prediction done, add your numbers now")
        self.add_your_numbers()

# Starting the class
FootballStylePredictor()


Type Object.start() to start the class


In [ ]:
player_selection = FootballStylePredictor()
player_selection.start('final_female.csv', 'gk_female.csv')
print(player_selection.predictions)